Подключение библиотек

In [40]:
import telebot
import os
import speech_recognition
from pydub import AudioSegment

Создание бота

In [41]:
token = '6874102761:AAFyjBU_JP1oFf47UkOQ-REQSZ2RqD0kodg'
bot = telebot.TeleBot(token)

Преобразование формата аудио .oga в .wav

In [42]:
def ogaTowav(filename):
    # Переименование формата 'sample.oga' в 'sample.wav'
    new_filename = filename.replace('.oga', '.wav')
    # Чтение файла с диска
    audio = AudioSegment.from_file(filename)
    # Экспорт файла в новом формате
    audio.export(new_filename, format='wav')
    # Возврат имени нового файла как результата функции
    return new_filename

Преобразование голоса в текст

In [43]:
def recognize_speech(oga_filename):
    wav_filename = ogaTowav(oga_filename)
    recognizer = speech_recognition.Recognizer()
    
    with speech_recognition.WavFile(wav_filename) as source:
        wav_audio = recognizer.record(source)
    
    text = recognizer.recognize_google(wav_audio, language='ru')
    
    # Удаление ненужных файлов с диска
    if os.path.exists(oga_filename):
        os.remove(oga_filename)
    if os.path.exists(wav_filename):
        os.remove(wav_filename)
    
    return text

Скачивание файла, который прислал пользователь

In [44]:
def download_file(bot, file_id):
    # Получение информации о файле:
    file_info = bot.get_file(file_id)

    # Загрузка файла с помощью функции bot.download_file
    # по информации о файле file_info.file_path
    downloaded_file = bot.download_file(file_info.file_path)

    # Создание уникального имени файла:
    filename = file_id + file_info.file_path

    # Имя файла имеет вид 'voice/file_123.oga', замена "/" на "_"
    filename = filename.replace('/', '_')

    with open(filename, 'wb') as f:
        f.write(downloaded_file)

    return filename

Функция приветствия

In [45]:
@bot.message_handler(commands=['start'])
def say_hello(message):
    username = message.from_user.first_name
    if username is None:
        username = message.from_user.username
    bot.send_message(message.chat.id, f'Здравствуй, {username}!')

Новое условие срабатывания на голосовое сообщение 

In [46]:
# Новое условие: content_types=['voice']
@bot.message_handler(content_types=['voice'])
def transcript(message):
    filename = download_file(bot, message.voice.file_id)
    
    # Распознание записи с помощью функции recognize_speech
    text = recognize_speech(filename)

    # Вывод текста в ответ пользователю
    bot.send_message(message.chat.id, text)

Запуск бота

In [47]:
bot.polling()